In [6]:
import pygame
import math
from collections import defaultdict

class HexTile(pygame.Surface):
    """Surface esagonale con palette/descrizione e supporto 2.5D"""
    def __init__(self, size, palette_description, pointy_top=True, **kwargs):
        self.hex_size = size
        self.pointy_top = pointy_top
        
        # Aumentiamo l'altezza per includere la banda laterale
        self.band_height = int(size / 4)
        
        if pointy_top:
            width = int(size * math.sqrt(3))
            height = int(size * 2) + self.band_height
        else:
            width = int(size * 2)
            height = int(size * math.sqrt(3)) + self.band_height
        
        super().__init__((width, height), pygame.SRCALPHA, **kwargs)
        self.palette = palette_description
        self.fill((0, 0, 0, 0))
    
    def get_hex_points_absolute(self, screen_x, screen_y):
        """Ritorna i punti dell'esagono in coordinate assolute dello schermo"""
        cx = screen_x + self.get_width() // 2
        cy = screen_y + self.hex_size
        size = self.hex_size
        points = []
        
        for i in range(6):
            if self.pointy_top:
                angle = math.pi / 3 * i - math.pi / 6
            else:
                angle = math.pi / 3 * i
            
            x = cx + size * math.cos(angle)
            y = cy + size * math.sin(angle)
            points.append((x, y))
        
        return points
    
    def draw_3d(self, screen, screen_x, screen_y):
        """Disegna il tile in modalità 2.5D: prima banda laterale, poi faccia top"""
        # 1. Disegna la banda laterale
        if self.band_height > 0:
            self._draw_band(screen, screen_x, screen_y)
        
        # 2. Disegna l'esagono top
        points = self.get_hex_points_absolute(screen_x, screen_y)
        pygame.draw.polygon(screen, self.palette['color'], points)
        pygame.draw.polygon(screen, (0, 0, 0), points, 2)
    
    def _draw_band(self, screen, screen_x, screen_y):
        """Disegna la banda laterale con colore più scuro (effetto ombra)"""
        base_color = self.palette['color']
        hex_points = self.get_hex_points_absolute(screen_x, screen_y)
        
        if self.pointy_top:
            # Lato SE (E-SE, vertici 0-1)
            dark_color1 = tuple(int(c * 0.70) for c in base_color)
            self._draw_band_face(screen, hex_points[0], hex_points[1], dark_color1)
            
            # Lato S (SE-SW, vertici 1-2)
            dark_color2 = tuple(int(c * 0.55) for c in base_color)
            self._draw_band_face(screen, hex_points[1], hex_points[2], dark_color2)
            
            # Lato SW (SW-W, vertici 2-3)
            dark_color3 = tuple(int(c * 0.70) for c in base_color)
            self._draw_band_face(screen, hex_points[2], hex_points[3], dark_color3)
    
    def _draw_band_face(self, screen, point1, point2, color):
        """Disegna una singola faccia della banda come quadrilatero"""
        p1_top = point1
        p2_top = point2
        p1_bottom = (point1[0], point1[1] + self.band_height)
        p2_bottom = (point2[0], point2[1] + self.band_height)
        
        quad = [p1_top, p2_top, p2_bottom, p1_bottom]
        pygame.draw.polygon(screen, color, quad)
        pygame.draw.polygon(screen, (0, 0, 0), quad, 1)


class Camera:
    """Gestisce la camera con transizioni smooth"""
    def __init__(self, screen_width, screen_height, transition_time=0.25):
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.transition_time = transition_time
        
        self.current_x = 0.0
        self.current_y = 0.0
        self.target_x = 0.0
        self.target_y = 0.0
        
        self.transition_progress = 1.0
    
    def set_target(self, target_x, target_y):
        """Imposta il nuovo target per la camera"""
        self.target_x = target_x
        self.target_y = target_y
        self.transition_progress = 0.0
    
    def update(self, dt):
        """Aggiorna la posizione della camera con interpolazione smooth"""
        if self.transition_progress < 1.0:
            self.transition_progress += dt / self.transition_time
            self.transition_progress = min(1.0, self.transition_progress)
            
            self.current_x += (self.target_x - self.current_x) * min(1.0, dt / self.transition_time * 4)
            self.current_y += (self.target_y - self.current_y) * min(1.0, dt / self.transition_time * 4)
    
    def get_offset(self):
        """Ritorna l'offset corrente della camera"""
        return self.current_x, self.current_y


class HexGrid:
    """Gestisce griglia esagonale 2.5D con sistema a LAYER basati su Z"""
    
    DIRECTIONS_POINTY = [
        (1, 0),   # E (Est)
        (1, -1),  # NE (Nord-Est)
        (0, -1),  # NW (Nord-Ovest)
        (-1, 0),  # W (Ovest)
        (-1, 1),  # SW (Sud-Ovest)
        (0, 1)    # SE (Sud-Est)
    ]
    
    def __init__(self, hex_size, palettes, pointy_top=True):
        self.hex_size = hex_size
        self.palettes = palettes
        self.pointy_top = pointy_top
        self.tiles = {}  # Chiave: (q, r, z)
        self.current_pos = (0, 0, 0)
        
        self.DIRECTIONS = self.DIRECTIONS_POINTY
        
        self.generate_tile(0, 0, 0)
    
    def axial_to_pixel(self, q, r, z):
        """Converte coordinate axial 3D (q, r, z) in coordinate pixel dello schermo"""
        if self.pointy_top:
            x = self.hex_size * math.sqrt(3) * (q + r/2)
            y_base = self.hex_size * 3/2 * r
        else:
            x = self.hex_size * 3/2 * q
            y_base = self.hex_size * math.sqrt(3) * (r + q/2)
        
        # Offset verticale per l'elevazione
        y = y_base - (z * self.hex_size / 4)
        
        return x, y
    
    def generate_tile(self, q, r, z):
        """Genera una tile alle coordinate (q, r, z) se non esiste già"""
        if (q, r, z) in self.tiles:
            return self.tiles[(q, r, z)]
        
        palette_index = (abs(q) + abs(r) + abs(z)) % len(self.palettes)
        palette = self.palettes[palette_index]
        
        tile = HexTile(self.hex_size, palette, pointy_top=self.pointy_top)
        
        self.tiles[(q, r, z)] = tile
        return tile
    
    def get_neighbor(self, q, r, direction_index):
        """Ottiene coordinate del vicino orizzontale nella direzione specificata"""
        dq, dr = self.DIRECTIONS[direction_index]
        return (q + dq, r + dr)
    
    def move_horizontal(self, direction_index):
        """Muove la posizione corrente in orizzontale (mantiene stesso z)"""
        q, r, z = self.current_pos
        new_q, new_r = self.get_neighbor(q, r, direction_index)
        self.current_pos = (new_q, new_r, z)
        self.generate_tile(new_q, new_r, z)
        return self.current_pos
    
    def move_up(self):
        """Muove verso l'alto (z+1) - crea tile sopra quella corrente"""
        q, r, z = self.current_pos
        new_z = z + 1
        self.current_pos = (q, r, new_z)
        self.generate_tile(q, r, new_z)
        return self.current_pos
    
    def move_down(self):
        """Muove verso il basso (z-1) - cancella tile corrente dopo lo spostamento"""
        q, r, z = self.current_pos
        new_z = z - 1
        
        self.generate_tile(q, r, new_z)
        
        if (q, r, z) in self.tiles:
            del self.tiles[(q, r, z)]
        
        self.current_pos = (q, r, new_z)
        return self.current_pos
    
    def generate_neighbors(self, q, r, z):
        """Genera tutte le tiles vicine a (q, r, z) allo stesso livello z"""
        for i in range(6):
            nq, nr = self.get_neighbor(q, r, i)
            self.generate_tile(nq, nr, z)
    
    def get_camera_target(self, screen_width, screen_height):
        """Calcola la posizione target della camera per centrare l'esagono corrente"""
        q, r, z = self.current_pos
        hex_x, hex_y = self.axial_to_pixel(q, r, z)
        
        camera_x = screen_width // 2 - hex_x
        camera_y = screen_height // 2 - hex_y
        
        return camera_x, camera_y
    
    def draw(self, screen, camera_x=0, camera_y=0):
        """
        Disegna usando sistema a LAYER basati su Z.
        Ogni livello Z viene renderizzato separatamente, dal più basso al più alto.
        All'interno di ogni layer, usa painter's algorithm 2D.
        """
        # Raggruppa tile per livello Z
        
        layers = defaultdict(list)
        for (q, r, z) in self.tiles.keys():
            layers[z].append((q, r, z))
        
        # Ordina i livelli Z dal più basso al più alto
        sorted_z_levels = sorted(layers.keys())
        
        # Renderizza ogni layer in ordine
        for z_level in sorted_z_levels:
            # All'interno del layer, ordina con painter's algorithm 2D
            # Usa (r, q) per griglia esagonale isometrica
            tiles_in_layer = sorted(layers[z_level], 
                                   key=lambda pos: (pos[1], pos[0]))
            
            # Disegna ogni tile del layer corrente
            for (q, r, z) in tiles_in_layer:
                tile = self.tiles[(q, r, z)]
                x, y = self.axial_to_pixel(q, r, z)
                
                screen_x = int(x + camera_x)
                screen_y = int(y + camera_y)
                
                # Disegna il tile 3D
                tile.draw_3d(screen, screen_x, screen_y)
                
                # Evidenzia tile corrente con bordo giallo
                if (q, r, z) == self.current_pos:
                    points = tile.get_hex_points_absolute(screen_x, screen_y)
                    pygame.draw.polygon(screen, (255, 255, 0), points, 4)


def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    pygame.display.set_caption("Hex Tiles - 2.5D con Sistema a Layer")
    clock = pygame.time.Clock()
    
    palettes = [
        {'name': 'Grass', 'color': (34, 139, 34), 'elevation': 0},
        {'name': 'Forest', 'color': (0, 100, 0), 'elevation': 1},
        {'name': 'Water', 'color': (30, 144, 255), 'elevation': -1},
        {'name': 'Mountain', 'color': (139, 137, 137), 'elevation': 3},
        {'name': 'Sand', 'color': (238, 214, 175), 'elevation': 0}
    ]
    
    hex_grid = HexGrid(40, palettes, pointy_top=True)
    camera = Camera(screen.get_width(), screen.get_height(), transition_time=0.25)
    
    initial_target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
    camera.current_x, camera.current_y = initial_target
    camera.target_x, camera.target_y = initial_target
    
    font = pygame.font.Font(None, 22)
    
    running = True
    dt = 0
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                
                # Movimento orizzontale (Q, E, A, D, S, X)
                elif event.key == pygame.K_q:  # NW
                    hex_grid.move_horizontal(2)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_e:  # NE
                    hex_grid.move_horizontal(1)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_a:  # W
                    hex_grid.move_horizontal(3)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_d:  # E
                    hex_grid.move_horizontal(0)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_s:  # SW
                    hex_grid.move_horizontal(4)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_x:  # SE
                    hex_grid.move_horizontal(5)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                
                # Movimento verticale (W, Z)
                elif event.key == pygame.K_w:  # Su (z+1)
                    hex_grid.move_up()
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_z:  # Giù (z-1)
                    hex_grid.move_down()
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                
                elif event.key == pygame.K_SPACE:
                    q, r, z = hex_grid.current_pos
                    hex_grid.generate_neighbors(q, r, z)
        
        camera.update(dt)
        camera_x, camera_y = camera.get_offset()
        
        screen.fill((200, 200, 200))
        hex_grid.draw(screen, camera_x, camera_y)
        
        instructions = [
            "=== SISTEMA A LAYER (Z-based) ===",
            "Q: NW  E: NE",
            "A: W   D: E",
            "S: SW  X: SE",
            "",
            "W: Sali layer (z+1)",
            "Z: Scendi layer (z-1)",
            "",
            "SPACE: Genera vicini",
            "ESC: Esci",
            "",
            f"Tiles totali: {len(hex_grid.tiles)}",
            f"Posizione (q,r,z): {hex_grid.current_pos}",
            f"Layer attivi: {len(set(z for q,r,z in hex_grid.tiles.keys()))}"
        ]
        
        for i, text in enumerate(instructions):
            surf = font.render(text, True, (0, 0, 0))
            screen.blit(surf, (10, 10 + i * 22))
        
        pygame.display.flip()
        dt = clock.tick(120) / 1000.0
    
    pygame.quit()

if __name__ == "__main__":
    main()

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1342:(snd_func_refer) error evaluating name
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5727:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM default


In [16]:
import pygame
import math
from collections import defaultdict

class HexTile(pygame.Surface):
    """Surface esagonale con palette/descrizione e supporto 2.5D con 7 sezioni"""
    def __init__(self, size, palette_description, pointy_top=True, **kwargs):
        self.hex_size = size
        self.pointy_top = pointy_top
        
        # Aumentiamo l'altezza per includere la banda laterale
        self.band_height = int(size / 4)
        
        if pointy_top:
            width = int(size * math.sqrt(3))
            height = int(size * 2) + self.band_height
        else:
            width = int(size * 2)
            height = int(size * math.sqrt(3)) + self.band_height
        
        super().__init__((width, height), pygame.SRCALPHA, **kwargs)
        self.palette = palette_description
        
        # Fattore di scala per l'esagono centrale (60% = 0.60)
        self.center_scale = 0.60
        
        self.fill((0, 0, 0, 0))
    
    def get_hex_points_absolute(self, screen_x, screen_y, scale=1.0):
        """Ritorna i punti dell'esagono in coordinate assolute dello schermo
        
        Args:
            screen_x: coordinata x sullo schermo
            screen_y: coordinata y sullo schermo
            scale: fattore di scala (1.0 = dimensione normale, 0.35 = esagono centrale)
        """
        cx = screen_x + self.get_width() // 2
        cy = screen_y + self.hex_size
        size = self.hex_size * scale
        points = []
        
        for i in range(6):
            if self.pointy_top:
                angle = math.pi / 3 * i - math.pi / 6
            else:
                angle = math.pi / 3 * i
            
            x = cx + size * math.cos(angle)
            y = cy + size * math.sin(angle)
            points.append((x, y))
        
        return points
    
    def get_trapezoid_points(self, center_points, outer_points, section_index):
        """Calcola i 4 vertici del trapezio per una sezione radiale
        
        Args:
            center_points: vertici dell'esagono centrale
            outer_points: vertici dell'esagono esterno
            section_index: indice della sezione (0-5)
        
        Returns:
            Lista di 4 punti che formano il trapezio
        """
        # Indice del vertice successivo (con wrap-around)
        next_index = (section_index + 1) % 6
        
        # Trapezio formato da:
        # - Due vertici consecutivi dell'esagono centrale
        # - Due vertici consecutivi dell'esagono esterno
        trapezoid = [
            center_points[section_index],      # Vertice centro 1
            center_points[next_index],         # Vertice centro 2
            outer_points[next_index],          # Vertice esterno 2
            outer_points[section_index]        # Vertice esterno 1
        ]
        
        return trapezoid
    
    def draw_3d(self, screen, screen_x, screen_y):
        """Disegna il tile in modalità 2.5D con 7 sezioni: prima banda laterale, poi faccia top"""
        # 1. Disegna la banda laterale
        if self.band_height > 0:
            self._draw_band(screen, screen_x, screen_y)
        
        # 2. Disegna la faccia top suddivisa in 7 sezioni
        self._draw_top_face(screen, screen_x, screen_y)
    
    def _draw_top_face(self, screen, screen_x, screen_y):
        """Disegna la faccia top suddivisa in esagono centrale + 6 trapezi"""
        # Calcola vertici esagono esterno (100%)
        outer_points = self.get_hex_points_absolute(screen_x, screen_y, scale=1.0)
        
        # Calcola vertici esagono centrale (35%)
        center_points = self.get_hex_points_absolute(screen_x, screen_y, scale=self.center_scale)
        
        # Ottieni colori dalla palette
        center_color = self.palette.get('center', (100, 100, 100))
        section_colors = self.palette.get('sections', [(150, 150, 150)] * 6)
        
        # Disegna i 6 trapezi radiali
        for i in range(6):
            trapezoid = self.get_trapezoid_points(center_points, outer_points, i)
            section_color = section_colors[i] if i < len(section_colors) else (150, 150, 150)
            
            # Disegna il trapezio
            pygame.draw.polygon(screen, section_color, trapezoid)
            # Bordo nero per distinguere le sezioni
            pygame.draw.polygon(screen, (0, 0, 0), trapezoid, 1)
        
        # Disegna l'esagono centrale sopra i trapezi
        pygame.draw.polygon(screen, center_color, center_points)
        pygame.draw.polygon(screen, (0, 0, 0), center_points, 1)
        
        # Bordo esterno dell'intero esagono
        pygame.draw.polygon(screen, (0, 0, 0), outer_points, 2)
    
    def _draw_band(self, screen, screen_x, screen_y):
        """Disegna la banda laterale con colore più scuro (effetto ombra)
        Usa il colore medio delle sezioni per la banda"""
        # Calcola colore medio per la banda (usa le sections, non il centro)
        section_colors = self.palette.get('sections', [(150, 150, 150)] * 6)
        
        # Media dei colori delle sezioni visibili (quelle sul fronte)
        if len(section_colors) >= 3:
            # Usa le 3 sezioni frontali (0, 1, 2) per calcolare il colore medio
            avg_r = sum(c[0] for c in section_colors[0:3]) // 3
            avg_g = sum(c[1] for c in section_colors[0:3]) // 3
            avg_b = sum(c[2] for c in section_colors[0:3]) // 3
            base_color = (avg_r, avg_g, avg_b)
        else:
            base_color = section_colors[0] if section_colors else (150, 150, 150)
        
        hex_points = self.get_hex_points_absolute(screen_x, screen_y, scale=1.0)
        
        if self.pointy_top:
            # Lato SE (E-SE, vertici 0-1)
            dark_color1 = tuple(int(c * 0.70) for c in base_color)
            self._draw_band_face(screen, hex_points[0], hex_points[1], dark_color1)
            
            # Lato S (SE-SW, vertici 1-2)
            dark_color2 = tuple(int(c * 0.55) for c in base_color)
            self._draw_band_face(screen, hex_points[1], hex_points[2], dark_color2)
            
            # Lato SW (SW-W, vertici 2-3)
            dark_color3 = tuple(int(c * 0.70) for c in base_color)
            self._draw_band_face(screen, hex_points[2], hex_points[3], dark_color3)
    
    def _draw_band_face(self, screen, point1, point2, color):
        """Disegna una singola faccia della banda come quadrilatero"""
        p1_top = point1
        p2_top = point2
        p1_bottom = (point1[0], point1[1] + self.band_height)
        p2_bottom = (point2[0], point2[1] + self.band_height)
        
        quad = [p1_top, p2_top, p2_bottom, p1_bottom]
        pygame.draw.polygon(screen, color, quad)
        pygame.draw.polygon(screen, (0, 0, 0), quad, 1)


class Camera:
    """Gestisce la camera con transizioni smooth"""
    def __init__(self, screen_width, screen_height, transition_time=0.25):
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.transition_time = transition_time
        
        self.current_x = 0.0
        self.current_y = 0.0
        self.target_x = 0.0
        self.target_y = 0.0
        
        self.transition_progress = 1.0
    
    def set_target(self, target_x, target_y):
        """Imposta il nuovo target per la camera"""
        self.target_x = target_x
        self.target_y = target_y
        self.transition_progress = 0.0
    
    def update(self, dt):
        """Aggiorna la posizione della camera con interpolazione smooth"""
        if self.transition_progress < 1.0:
            self.transition_progress += dt / self.transition_time
            self.transition_progress = min(1.0, self.transition_progress)
            
            self.current_x += (self.target_x - self.current_x) * min(1.0, dt / self.transition_time * 4)
            self.current_y += (self.target_y - self.current_y) * min(1.0, dt / self.transition_time * 4)
    
    def get_offset(self):
        """Ritorna l'offset corrente della camera"""
        return self.current_x, self.current_y


class HexGrid:
    """Gestisce griglia esagonale 2.5D con sistema a LAYER basati su Z"""
    
    DIRECTIONS_POINTY = [
        (1, 0),   # E (Est)
        (1, -1),  # NE (Nord-Est)
        (0, -1),  # NW (Nord-Ovest)
        (-1, 0),  # W (Ovest)
        (-1, 1),  # SW (Sud-Ovest)
        (0, 1)    # SE (Sud-Est)
    ]
    
    def __init__(self, hex_size, palettes, pointy_top=True):
        self.hex_size = hex_size
        self.palettes = palettes
        self.pointy_top = pointy_top
        self.tiles = {}  # Chiave: (q, r, z)
        self.current_pos = (0, 0, 0)
        
        self.DIRECTIONS = self.DIRECTIONS_POINTY
        
        self.generate_tile(0, 0, 0)
    
    def axial_to_pixel(self, q, r, z):
        """Converte coordinate axial 3D (q, r, z) in coordinate pixel dello schermo"""
        if self.pointy_top:
            x = self.hex_size * math.sqrt(3) * (q + r/2)
            y_base = self.hex_size * 3/2 * r
        else:
            x = self.hex_size * 3/2 * q
            y_base = self.hex_size * math.sqrt(3) * (r + q/2)
        
        # Offset verticale per l'elevazione
        y = y_base - (z * self.hex_size / 4)
        
        return x, y
    
    def generate_tile(self, q, r, z):
        """Genera una tile alle coordinate (q, r, z) se non esiste già"""
        if (q, r, z) in self.tiles:
            return self.tiles[(q, r, z)]
        
        palette_index = (abs(q) + abs(r) + abs(z)) % len(self.palettes)
        palette = self.palettes[palette_index]
        
        tile = HexTile(self.hex_size, palette, pointy_top=self.pointy_top)
        
        self.tiles[(q, r, z)] = tile
        return tile
    
    def get_neighbor(self, q, r, direction_index):
        """Ottiene coordinate del vicino orizzontale nella direzione specificata"""
        dq, dr = self.DIRECTIONS[direction_index]
        return (q + dq, r + dr)
    
    def move_horizontal(self, direction_index):
        """Muove la posizione corrente in orizzontale (mantiene stesso z)"""
        q, r, z = self.current_pos
        new_q, new_r = self.get_neighbor(q, r, direction_index)
        self.current_pos = (new_q, new_r, z)
        self.generate_tile(new_q, new_r, z)
        return self.current_pos
    
    def move_up(self):
        """Muove verso l'alto (z+1) - crea tile sopra quella corrente"""
        q, r, z = self.current_pos
        new_z = z + 1
        self.current_pos = (q, r, new_z)
        self.generate_tile(q, r, new_z)
        return self.current_pos
    
    def move_down(self):
        """Muove verso il basso (z-1) - cancella tile corrente dopo lo spostamento"""
        q, r, z = self.current_pos
        new_z = z - 1
        
        self.generate_tile(q, r, new_z)
        
        if (q, r, z) in self.tiles:
            del self.tiles[(q, r, z)]
        
        self.current_pos = (q, r, new_z)
        return self.current_pos
    
    def generate_neighbors(self, q, r, z):
        """Genera tutte le tiles vicine a (q, r, z) allo stesso livello z"""
        for i in range(6):
            nq, nr = self.get_neighbor(q, r, i)
            self.generate_tile(nq, nr, z)
    
    def get_camera_target(self, screen_width, screen_height):
        """Calcola la posizione target della camera per centrare l'esagono corrente"""
        q, r, z = self.current_pos
        hex_x, hex_y = self.axial_to_pixel(q, r, z)
        
        camera_x = screen_width // 2 - hex_x
        camera_y = screen_height // 2 - hex_y
        
        return camera_x, camera_y
    
    def draw(self, screen, camera_x=0, camera_y=0):
        """
        Disegna usando sistema a LAYER basati su Z.
        Ogni livello Z viene renderizzato separatamente, dal più basso al più alto.
        All'interno di ogni layer, usa painter's algorithm 2D.
        """
        # Raggruppa tile per livello Z
        layers = defaultdict(list)
        for (q, r, z) in self.tiles.keys():
            layers[z].append((q, r, z))
        
        # Ordina i livelli Z dal più basso al più alto
        sorted_z_levels = sorted(layers.keys())
        
        # Renderizza ogni layer in ordine
        for z_level in sorted_z_levels:
            # All'interno del layer, ordina con painter's algorithm 2D
            # Usa (r, q) per griglia esagonale isometrica
            tiles_in_layer = sorted(layers[z_level], 
                                   key=lambda pos: (pos[1], pos[0]))
            
            # Disegna ogni tile del layer corrente
            for (q, r, z) in tiles_in_layer:
                tile = self.tiles[(q, r, z)]
                x, y = self.axial_to_pixel(q, r, z)
                
                screen_x = int(x + camera_x)
                screen_y = int(y + camera_y)
                
                # Disegna il tile 3D
                tile.draw_3d(screen, screen_x, screen_y)
                
                # Evidenzia tile corrente con bordo giallo
                if (q, r, z) == self.current_pos:
                    points = tile.get_hex_points_absolute(screen_x, screen_y, scale=1.0)
                    pygame.draw.polygon(screen, (255, 255, 0), points, 4)


def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    pygame.display.set_caption("Hex Tiles - 2.5D con 7 Sezioni")
    clock = pygame.time.Clock()
    
    # Palette con 7 colori: 1 centro + 6 sezioni radiali
    palettes = [
        {
            'name': 'Grass Mixed',
            'center': (34, 139, 34),  # Verde erba centrale
            'sections': [
                (50, 155, 50),   # E - Verde chiaro
                (34, 139, 34),   # NE - Verde erba
                (20, 120, 20),   # NW - Verde scuro
                (40, 150, 40),   # W - Verde medio
                (30, 130, 30),   # SW - Verde
                (45, 145, 45)    # SE - Verde chiaro
            ]
        },
        {
            'name': 'Forest Varied',
            'center': (0, 100, 0),
            'sections': [
                (10, 110, 10),
                (0, 90, 0),
                (5, 105, 5),
                (0, 95, 0),
                (15, 115, 15),
                (0, 100, 0)
            ]
        },
        {
            'name': 'Water Deep',
            'center': (30, 144, 255),
            'sections': [
                (40, 154, 255),
                (25, 139, 245),
                (35, 149, 250),
                (30, 144, 255),
                (20, 134, 240),
                (45, 159, 255)
            ]
        },
        {
            'name': 'Mountain Rocky',
            'center': (139, 137, 137),
            'sections': [
                (149, 147, 147),
                (129, 127, 127),
                (144, 142, 142),
                (134, 132, 132),
                (154, 152, 152),
                (139, 137, 137)
            ]
        },
        {
            'name': 'Sand Beach',
            'center': (238, 214, 175),
            'sections': [
                (248, 224, 185),
                (228, 204, 165),
                (243, 219, 180),
                (233, 209, 170),
                (253, 229, 190),
                (238, 214, 175)
            ]
        },
        {
            'name': 'Mixed Terrain',
            'center': (100, 200, 100),
            'sections': [
                (150, 150, 50),   # Giallo
                (200, 100, 100),  # Rosso
                (100, 100, 200),  # Blu
                (150, 200, 150),  # Verde chiaro
                (200, 200, 100),  # Giallo-verde
                (100, 150, 200)   # Azzurro
            ]
        }
    ]
    
    hex_grid = HexGrid(120, palettes, pointy_top=True)
    camera = Camera(screen.get_width(), screen.get_height(), transition_time=0.25)
    
    initial_target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
    camera.current_x, camera.current_y = initial_target
    camera.target_x, camera.target_y = initial_target
    
    font = pygame.font.Font(None, 22)
    
    running = True
    dt = 0
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                
                # Movimento orizzontale (Q, E, A, D, S, X)
                elif event.key == pygame.K_q:  # NW
                    hex_grid.move_horizontal(2)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_e:  # NE
                    hex_grid.move_horizontal(1)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_a:  # W
                    hex_grid.move_horizontal(3)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_d:  # E
                    hex_grid.move_horizontal(0)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_s:  # SW
                    hex_grid.move_horizontal(4)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_x:  # SE
                    hex_grid.move_horizontal(5)
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                
                # Movimento verticale (W, Z)
                elif event.key == pygame.K_w:  # Su (z+1)
                    hex_grid.move_up()
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                elif event.key == pygame.K_z:  # Giù (z-1)
                    hex_grid.move_down()
                    target = hex_grid.get_camera_target(screen.get_width(), screen.get_height())
                    camera.set_target(*target)
                
                elif event.key == pygame.K_SPACE:
                    q, r, z = hex_grid.current_pos
                    hex_grid.generate_neighbors(q, r, z)
        
        camera.update(dt)
        camera_x, camera_y = camera.get_offset()
        
        screen.fill((200, 200, 200))
        hex_grid.draw(screen, camera_x, camera_y)
        
        instructions = [
            "=== HEX TILES - 7 SEZIONI ===",
            "Q: NW  E: NE",
            "A: W   D: E",
            "S: SW  X: SE",
            "",
            "W: Sali layer (z+1)",
            "Z: Scendi layer (z-1)",
            "",
            "SPACE: Genera vicini",
            "ESC: Esci",
            "",
            f"Tiles totali: {len(hex_grid.tiles)}",
            f"Posizione (q,r,z): {hex_grid.current_pos}",
            f"Layer attivi: {len(set(z for q,r,z in hex_grid.tiles.keys()))}"
        ]
        
        for i, text in enumerate(instructions):
            surf = font.render(text, True, (0, 0, 0))
            screen.blit(surf, (10, 10 + i * 22))
        
        pygame.display.flip()
        dt = clock.tick(120) / 1000.0
    
    pygame.quit()

if __name__ == "__main__":
    main()

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1342:(snd_func_refer) error evaluating name
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5727:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2721:(snd_pcm_open_noupdate) Unknown PCM default
